# 3.YoLoV2 未完
YoLoV1相对于Fast R-CNN，YOLO在目标定位方面错误率较高，YoLoV2的改进集中于在保持分类准确率的基础上增强定位精确度

## 3.1.YOLOV2的改进
### 3.1.1.全新网络Darknet-19
YOLOv2采用了全新的Darknet-19网络，将YOLOv1中最后的全连接层换成了卷积层。最后的特征图大小为$13 \times 13$.输入为$416 \times 416$

### 3.1.2.Anchors Box
采纳了Faster RCNN中的anchors box的思想，每个位置的各个anchor box都单独预测一套分类概率。anchors box数量为5。为何要引入anchors呢，因为在anchors上微调比直接去回归坐标要容易得多。

## 3.2.损失函数
YOLOv2的损失函数比YOLOv1的更复杂。
$$\begin{eqnarray}
Loss_t=\sum_{i=0}^W\sum_{j=0}^H\sum_{k=0}^A& &1_{MaxIOU \le Thresh}\lambda_{noobj} \times (-b_{ijk}^o)^2\\
&+&1_{t \le 12800}\lambda_{prior} \times \sum_{r \in (x,y,W,h)}(prior_k^r-b_{ijk}^r)^2 \\
&+&1_k^{truth}[\lambda_{coord} \times \sum_{r \in (x,y,W,h)}(truth^r-b_{ijk}^r)^2]\\
&+&1_k^{truth}[\lambda_{obj} \times (IOU_{truth}^k-b_{ijk}^o)^2]\\
&+&1_k^{truth}[\lambda_{class} \times \sum_{c=1}^C(truth^c-b_{ijk}^c)^2]
\end{eqnarray}$$

## 3.3.损失函数释义
$W,H$分别指的是特征图的宽和高，分别是13,13，A指的是先验框的数目5。

### 3.3.1.第一项
$$1_{MaxIOU \le Thresh}\lambda_{noobj} \times (-b_{ijk}^o)^2$$
loss是计算background的置信度误差，判断是否是用来预测背景的逻辑就是$1_{MaxIOU \le Thresh}$。需要先计算各个预测框和所有ground truth的IOU值，并且取最大值Max_IOU，如果该值小于一定的阈值(0.6)，那么这个预测框就标记为background，需要计算noobj的置信度误差

### 3.3.2.第二项
计算先验框与预测宽的坐标误差，但是只在前12800上计算，这项应该是在训练前期使预测框快速学习到先验框的形状。

### 3.3.3.第三项
计算与某个ground truth匹配的预测框各部分的坐标误差

### 3.3.4.第四项
计算与某个ground truth匹配的预测框各部分的置信度误差

### 3.3.5.第五项
计算与某个ground truth匹配的预测框各部分的分类误差
